<a href="https://colab.research.google.com/github/bhoomika297/Langchain-projects/blob/main/Langchain_Basic_Part1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install langchain langchain_groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.9 MB/s eta 0:00:00


### Setting environment variable to start logging trace

In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["langsmith_api_key"] = getpass.getpass()

··········


In [10]:
os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-8b-8192")

··········


In [11]:
from langchain_core.messages import HumanMessage , SystemMessage

messages = [
    SystemMessage(content="Translate the following text from english into Hindi"),
    HumanMessage(content="Hello, how are you?"),
]

model.invoke(messages)

AIMessage(content='नमस्ते, तुम कैसे हो? (Namaste, tum kaise ho?)', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 29, 'total_tokens': 52, 'completion_time': 0.019166667, 'prompt_time': 0.004743213, 'queue_time': 0.07542127600000001, 'total_time': 0.02390988}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-60658318-07c1-46c5-8935-2f3c937e27de-0', usage_metadata={'input_tokens': 29, 'output_tokens': 23, 'total_tokens': 52})

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [13]:
result = model.invoke(messages)

In [14]:
parser.invoke(result)

'नमस्ते, तुम कैसे हो? (Namaste, tum kaise ho?)'

## Using Chaining method

In [15]:
# First Language model called and then it's output is passed through the parser

chain = model|parser        #  model----->parser

In [16]:
chain.invoke(messages)

'नमस्ते, तुम कैसे हो? (Namaste, tum kaise ho?)'

# PROMPT TEMPLATES

PromptTemplates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

In [17]:
from langchain_core.prompts import ChatPromptTemplate

In [18]:
system_template = "Translate the following into {language}:"

In [19]:
prompt_template = ChatPromptTemplate(
    [('system',system_template),('user',"{text}")]
)

In [20]:
result = prompt_template.invoke({'language':'Hindi','text':'Hi, this is bhoomika agrahari'})

result.to_messages()

[SystemMessage(content='Translate the following into Hindi:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi, this is bhoomika agrahari', additional_kwargs={}, response_metadata={})]

In [21]:
chain = prompt_template | model | parser       # prompt_template(message formating)-------->model-------->parser

In [22]:
chain.invoke({'language':'hindi','text':'Hi, I am bhoomika agrahari'})

'नमस्ते, मैं भूमिका अग्रहरी हूँ'

In [25]:
pip install langserve

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.8 MB/s eta 0:00:00


In [23]:
pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.9 MB/s eta 0:00:00


In [26]:
pip install uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.8 MB/s eta 0:00:00


IMPLEMENTING LANGSERVER TO CREATE SERVER FOR THE APPLICATION

In [30]:
from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langserve import add_routes

system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
     ('user','{text}')
    ])


model = ChatGroq(model="llama3-8b-8192")

parser = StrOutputParser()

chain = prompt_template | model | parser

app = FastAPI(
    title = "Langchain Server",
    version = "1.0",
    description = "A simple API server using langchain's runnable interfaces",
)

add_routes(app,
           chain,
           path = "/chain",
           )

if __name__ == "__main__":
  import uvicorn

  uvicorn.run(app, host="localhost", port=8000)



RuntimeError: asyncio.run() cannot be called from a running event loop

In [29]:
pip install sse_starlette